Purpose: Playing around with NB Multinormial language modeling. Eventually need to turn this into a .py file.

In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [36]:
df = pickle.load(open("../movies.p", 'rb'))
#df = pickle.load(open("../data/movies_text.p", 'rb'))

In [38]:
# remove unknown gender. 0 is male and 1 is female.
df = df[df.gender_from != '?']
df['gender_from'] = np.where(df.gender_from == 'f', 1, 0)

C:\Users\JoanWang\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [39]:
# turn list into a string
df['words'] = df['words'].map(lambda x: ' '.join(x))

C:\Users\JoanWang\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [40]:
df.head()
df.shape

,gender_to,movie_id,gender_from,char_id_from,char_id_to,line_id,words,movie_year,genre
0,m,m0,1,u0,u2,L194,we make quick roxanne korrine andrew barrett i...,1999,comedy
1,m,m0,1,u0,u2,L195,well i think we start pronunciation okay you,1999,comedy
2,m,m0,1,u0,u2,L196,hacking gagging spit part please,1999,comedy
3,m,m0,1,u0,u2,L197,okay bout we try french cuisine saturday night,1999,comedy
4,m,m0,1,u0,u2,L198,you ask me cute your name,1999,comedy


(243739, 9)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(df.words, df.gender_from.astype('int'), test_size=0.33, random_state=42)

In [23]:
y_train.value_counts()
y_test.value_counts()

0    114463
1     48842
Name: gender_from, dtype: int64

0    56246
1    24188
Name: gender_from, dtype: int64

In [24]:
X_train.shape
X_test.shape

(163305,)

(80434,)

In [25]:
# transform
count_vect = CountVectorizer(ngram_range=(1,2))
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [27]:
# look at sample of ngrams
count_vect.get_feature_names()[-50:]

['zug',
 'zuma',
 'zuma fantastic',
 'zundinger',
 'zundinger magician',
 'zur',
 'zur gansalt',
 'zuria',
 'zurich',
 'zurich yesterday',
 'zurich your',
 'zuul',
 'zuul idea',
 'zuul literature',
 'zuul refers',
 'zuzu',
 'zuzu matter',
 'zuzu my',
 'zuzu order',
 'zuzu people',
 'zuzu petal',
 'zuzu quiet',
 'zuzu zuzu',
 'zwoelf',
 'zwoelf uhr',
 'zyprexa',
 'zyprexa atavan',
 'zzzzzzzzzzzzzzzz',
 'zzzzzzzzzzzzzzzz thing',
 '³war',
 '³war hhh',
 'ßoor',
 'ßoor folk',
 'àquelqu',
 'àquelqu un',
 'ça',
 'ça alors',
 'ça pourrait',
 'élysées',
 'élysées adam',
 'éysée',
 'éysée seven',
 'êtes',
 'êtes certain',
 'ïretirement',
 'ïretirement hell',
 'ôcause',
 'ôcause back',
 'ôer',
 'ôer go']

In [26]:
# train
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [28]:
# test
X_test_counts = count_vect.transform(X_test) # JW TO DO: how does this treat unseen ngrams?
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

predicted = clf.predict(X_test_tfidf)
proba = clf.predict_proba(X_test_tfidf)

Classifier doesn't do well; predicting almost everything to be male :(

In [29]:
np.bincount(predicted)
predicted
proba

array([80016,   418], dtype=int64)

array([0, 0, 0, ..., 0, 0, 0])

array([[0.81486549, 0.18513451],
       [0.78756658, 0.21243342],
       [0.70091546, 0.29908454],
       ...,
       [0.70478833, 0.29521167],
       [0.78763772, 0.21236228],
       [0.91174784, 0.08825216]])